In [57]:
from docx import Document
import pandas as pd
import os
import re
import csv

In [58]:
'''
Set output file path
'''
########################################################################
out_filepath = '/Users/yangyi/Documents/CMU-S18/capstone/untitled folder/'
out_filename = 'Output_search.csv'
out_path = out_filepath + out_filename
########################################################################


header = ['File Name', 'Keywords', 'Change of Control'] # Column names in output file 

'''
Open the file  and set the header of columns in the output file
'''
with open (out_path, 'wb') as f:    
    wr = csv.writer(f)
    wr.writerow(header)


In [59]:
def contains_sliding_window(text, terms, win_len):
    target_words = re.split('[\s]', terms)
    probe = target_words[0]
    #print probe
    
    words = re.split('[\s]', text)
    flag = False
    for i in range(len(words)):
        word = words[i]
        if word.lower() != probe:
            continue
        l_pointer = max(0,i - win_len)
        r_pointer = min(len(words),i + win_len)
        for target_word in target_words:
            flag = False
            for pointer in range(l_pointer, r_pointer):
                if words[pointer] == target_word:
                    flag = True
            if not flag:
                break
        if flag:
            #print True
            return True
    #print False
    return False
        

In [60]:
#contains_sliding_window('hahahahah change of change jjj\'jjj\t control of change jjjj', 'change of of o f control', 3)

In [61]:
def get_bullet_pattern(text):
    pattern = None
    flag1 = re.match("\([A-Z]+\)", paras[j].text[:40].lstrip())
    flag2 = re.match("\([a-z]+\)", paras[j].text[:40].lstrip())
    flag3 = re.match("\([0-9]+\)", paras[j].text[:40].lstrip())
              
    if flag1:
        pattern = "\([A-Z]+\)"
    elif flag2:
        pattern = "\([a-z]+\)"
    elif flag3:
        pattern = "\([0-9]+\)"
    else:
        pattern = None
    return pattern
                                

In [62]:
   
'''
Open the output file in append mode and pull related language from the files in the input folder
'''
with open(out_path, 'ab') as f:
    
    # Input folder path
    directory_path = '/Users/yangyi/Documents/CMU-S18/capstone/untitled folder/Input Search files' 
    
    # Parse through each file in the input folder path
    for files in os.listdir(directory_path):
        print files
        
        # Create a Document object of each of the files
        document = Document(directory_path + "/" +files)
        
        # A list to add all the paragraphs in the document
        paras = []
        
        '''
        The following for loop is used to add all the paragraphs in the document to the paras list object
        '''
        for p in document.paragraphs:        
            paras.append(p)
            
        paras_number = len(paras) # Find the number of paragraphs in the document
        print paras_number
        
        paras_toPrint = [] # A list to add all the paragraphs relevant to 'Change of Control' language in this case
        
        for p in paras:    # Traversing through each of the paragraphs
            
            '''
            Assumption made : If length of the paragraph is less than 300 characters, it is probably the heading.
            I considered 300 characters, because I saw instances with a lot of whitespace characters before the characters 
            in heading.
            For Example:
            
            ....Law”, regardless of the date enacted, adopted or issued.
            
                            “Change of Control” means an event or series of events by which:
            (a)any “person” or “group” (as such terms are used in Sections 13(d) and 14(d) 
            of the Exchange Act, but excluding any employee benefit plan of such person or its 
            subsidiaries, and any person or entity acting in its capacity as trustee, agent or other fiduciary 
            or administrator of any such plan and any Initial Shareholder) becomes 

            
            '''
            
            if len(p.text) < 300: # Considering that this paragraph just pulls the heading
                #if contains_sliding_window(p.text[:100].lower(), 'change control', 3):
                if 'change of control' in p.text[:100].lower():
                #    considering Change of control is in the first 100 character of the heading

                    paras_toPrint.append(p) # Add the para in the paras_toPrint list
                    
                    
                    for j in range((paras.index(p) + 1), paras_number): 
                        # Loop from the paragraph after the heading containing 'Change of Control' to the last paragraph 
                        
                        if len(paras[j].text) < 10: # Ignore blank paragraphs as well as paragraphs containing page numbers and such
                            # print paras[j].text
                            continue
                        else:

                            match_flag = re.match("\([A-Za-z0-9]+\)",paras[j].text[:40].lstrip())
                            
                            if match_flag:                                 
                                paras_toPrint.append(paras[j])
                            else:
                                break        
            else: # Considering that this paragraph is not a heading
                #if contains_sliding_window(p.text.lower(), 'change control', 3):
                if 'change of control' in p.text.lower(): # Append to the list if change of control is in the list
                    paras_toPrint.append(p)
        wr = csv.writer(f)
        
        # Created a single list of all instances of 'Change of Control'
        # Ignored unrecognized ascii characters that throw an encoding error. Need improvement
        text =  [paras.text.encode('ascii','ignore') for paras in paras_toPrint]
        res = "".join(text)
        
        # write to file
        wr.writerow([files, 'Change of Control', res])
f.close()

Law Insider Contract 1oEdosIJEShqS79ujBWrpr.docx
2004
Law Insider Contract 39EVgZSlGahp2hBqZP8iXh.docx
3202
Law Insider Contract 5bhKmzEzgSf1XDXqsjd8wD.docx
1265
Law Insider Contract 6TabeU9bZWiWnaGtnjt8t6.docx
2199
Law Insider Contract rwfrbXnWOknr8FTAFaGzd.docx
48
Law Insider Contract z8Pmm0xF8ppq74uAwl83W.docx
313
